<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_TensorFlow/blob/main/14GPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

Computing Devices

In [2]:
def cpu():
    """Get the CPU device."""
    return tf.device('/CPU:0')
def gpu(i=0):
    """Get a GPU device."""
    return tf.device(f'/GPU:{i}')
cpu(), gpu(), gpu(1)

(<tensorflow.python.eager.context._EagerDeviceContext at 0x784ae1349680>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x784ae1349080>)

In [3]:
def num_gpus():
    """Get the number of available GPUs."""
    return len(tf.config.experimental.list_physical_devices('GPU'))
num_gpus()

1

In [4]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus():
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(<tensorflow.python.eager.context._EagerDeviceContext at 0x784adf877500>,
 [<tensorflow.python.eager.context._EagerDeviceContext at 0x784adf8891c0>])

Tensors and GPUs

In [5]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:GPU:0'

Storage on the GPU

In [6]:
with try_gpu():
    X = tf.ones((2, 3))
X

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [7]:
with try_gpu(1):
    Y = tf.random.uniform((2, 3))
Y

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.8367115 , 0.67943406, 0.30857766],
       [0.4115064 , 0.06240499, 0.8280138 ]], dtype=float32)>

In [8]:
with try_gpu(1):
    Z = X
print(X)
print(Z)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)


In [9]:
Y + Z

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.8367115, 1.6794341, 1.3085777],
       [1.4115064, 1.062405 , 1.8280138]], dtype=float32)>

In [10]:
with try_gpu(1):
    Z2 = Z
Z2 is Z

True

Neural Networks and GPUs

In [11]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1)])

In [12]:
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.17285645],
       [-0.17285645]], dtype=float32)>